# Ejercicio Aplicado

Colsubsidio ha dividido la prestación de sus servicios en once diferentes unidades especializadas de servicio (UES). Una de ellas, la UES de Droguerías, es la encargada de atender las necesidades de la población, afiliados y no afiliados a la caja a través de la red de droguerías a nivel nacional en cuanto a medicamentos, dispositivos médicos, fórmulas nutricionales y demás insumos. 

Se tienen las siguientes bases de datos:

1. **Afiliados vigentes**: Datos de las personas con afliación vigente a la caja
1. **Demograficos**: Descripcion socio-demográfica de los afiliados
1. **Empleador**: Datos de la empresa aportante.
1. **Familiar**: Informacion del grupo familiar de los afiliados
1. **Medicamentos**: Bases de datos del consumos por año de las droguerías comerciales de Colsubsidio

Las bases se relacionan de la siguiente manera:

 ![diagrama ER](images/ERdiagram.png "Optional title")
 
 Se plantea entrenar dos modelos:
 
 1. Calculo de la probabilidad que un afiliado compre o no en la drogueria
 2. Estimcaion del gasto anual en medicamentos de los afiliados

In [ ]:
import pandas as pd 
import numpy  as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
sns.set_palette("GnBu_d")
sns.set_style('whitegrid')

## 0. Cargue y depuracion de Datos

### Cargue de datos

In [ ]:
# Afiliado
afiliado = pd.read_csv("data/Afiliados.csv", encoding="latin", sep="|")
afiliado.head()

In [ ]:
# Demograficos
demograficos = pd.read_csv("data/Demograficos.csv", encoding="latin", sep=";")
demograficos.head()

In [ ]:
# Empleador
empleador = pd.read_csv("data/Empleador.csv", encoding="latin")
empleador.head()

In [ ]:
# Familiar
familiar = pd.read_excel("data/Familiar.xlsx", encoding="latin")
familiar.head()

In [ ]:
# Medicamentos
medicamentos = pd.read_table("data/Medicamentos2018.txt", encoding="latin")
medicamentos.head()

### Depuracion de Datos

In [ ]:
df1 = pd.merge(afiliado, demograficos, on='IdPersona', how='left')
df1.head()

In [ ]:
df2 = pd.merge(df1, familiar, on='IdPersona', how='left')
df2.head()

In [ ]:
persona = pd.merge(df2, empleador, on='IdEmpresa', how='left')
persona.head()

In [ ]:
persona.info()

#### Consumo Medicamentos Afiliado

Crea una base para identificar los afiliados que consumieron en la drogueria en el 2018

In [ ]:
medicamentos['Consume'] = 1
consumo_medicamentos=pd.DataFrame(medicamentos.groupby(['IdPersona','Consume'])['Venta_Neta'].sum().reset_index())

In [ ]:
data_afiliados = pd.merge(persona, consumo_medicamentos, on='IdPersona', how='left')

#Si el afilido no cruza en el consumo se asigna como cero en la variable indicadora y en el consumo
data_afiliados['Consume'].fillna(0, inplace=True)
data_afiliados['Venta_Neta'].fillna(0, inplace=True)

In [ ]:
data_afiliados.head()

In [ ]:
del afiliado, demograficos, empleador, familiar, medicamentos, consumo_medicamentos, df1, df2 

## 1. Pre-procesamiento de Datos

### Eliminacion de variables innecesarias

In [ ]:
data_afiliados.columns

In [ ]:
data_afiliados.drop(['Piramide1_y', 'Piramide2_y', 'CIIU', 'ActividadCIIU', 'SeccionCIIU',
                     'DivisionCIIU', 'GrupoCIIU', 'DescripcionCIIU', 'MunicipioEmpresa', 'LocalidadEmpresa',
                     'BarrioEmpresa'], axis=1, inplace=True)

In [ ]:
data_afiliados.columns

### Definicion del rol de Variables

In [ ]:
# Variables Categoricas
categoricas=['Categoria', 'Piramide1_x', 'Piramide2_x', 'Genero',
            'SectorPoblacional', 'estado_civil', 'DepartamentoPersona', 'MunicipioPersona',
            'segmento_grupo_familiar', 'DepartamentoEmpresa', 'SectorCIIU']

for var in categoricas:
    data_afiliados[var] = data_afiliados[var].astype("category")

In [ ]:
# Variable Objetivo
data_afiliados['Consume'] = data_afiliados['Consume'].astype("int64")

In [ ]:
data_afiliados.dtypes

## 1. Descripción de Datos

In [ ]:
sns.countplot(x="Categoria", data=data_afiliados)

In [ ]:
sns.countplot(x="Piramide1_x", data=data_afiliados)

In [ ]:
sns.countplot(x="Genero", data=data_afiliados)

In [ ]:
sns.countplot(x="SectorPoblacional", data=data_afiliados)

In [ ]:
sns.countplot(x="estado_civil", data=data_afiliados)

In [ ]:
sns.countplot(x="DepartamentoPersona", data=data_afiliados)

In [ ]:
sns.countplot(x="SectorCIIU", data=data_afiliados)

In [ ]:
sns.jointplot(x='Venta_Neta',y='Edad',data=data_afiliados)

In [ ]:
# sns.pairplot(data_afiliados, diag_kind ="kde")

### Matriz de Correlaciones

In [ ]:
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
sns.heatmap(data_afiliados.corr(), cmap="YlGnBu", square=True,annot=True)

### Diagramas de Cajas para variables categóricas

In [ ]:
fig , axes = plt.subplots(nrows=2,ncols=2)
fig.set_size_inches(10, 10)
sns.boxplot(data=data_afiliados,y="Venta_Neta",x='Categoria',orient="v",showfliers = False, ax=axes[0][0])
sns.boxplot(data=data_afiliados,y="Venta_Neta",x='Piramide1_x',orient="v", showfliers = False, ax=axes[1][0])
sns.boxplot(data=data_afiliados,y="Venta_Neta",x='segmento_grupo_familiar',orient="v", showfliers = False, ax=axes[1][1])
sns.boxplot(data=data_afiliados,y="Venta_Neta",x='SectorPoblacional',orient="v",showfliers = False, ax=axes[0][1])

### Descricpcion de las variables Objetivo

In [ ]:
sns.countplot(x="Consume", data=data_afiliados)

In [ ]:
sns.distplot(data_afiliados["Venta_Neta"] ,bins=50)

In [ ]:
fig,axes = plt.subplots(ncols=2,nrows=1)
fig.set_size_inches(12, 10)
sns.distplot(data_afiliados["Venta_Neta"],ax=axes[0])
stats.probplot(data_afiliados["Venta_Neta"], dist='norm', fit=True, plot=axes[1])

## 1. Preprocesamiento

In [ ]:
data_afiliados.isna().sum()

In [ ]:
data_afiliados.columns[data_afiliados.isna().any()].tolist()

### Imputar datos Numericos

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(data_afiliados[['cx_persona',  'cy_persona', 'EstratoPersona', 'cx_empresa', 'cy_empresa']])
data_afiliados[['cx_persona',  'cy_persona', 'EstratoPersona', 
                'cx_empresa', 'cy_empresa']]=pd.DataFrame(imp.transform(data_afiliados[['cx_persona', 'cy_persona', 'EstratoPersona',
                                                                                       'cx_empresa', 'cy_empresa']]))
data_afiliados.head()

In [ ]:
data_afiliados.columns[data_afiliados.isna().any()].tolist()

### Imputar datos Categoricos

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp.fit(data_afiliados[['DepartamentoPersona', 'MunicipioPersona', 'DepartamentoEmpresa']])
data_afiliados[['DepartamentoPersona', 'MunicipioPersona', 
                'DepartamentoEmpresa']]=pd.DataFrame(imp.transform(data_afiliados[['DepartamentoPersona', 'MunicipioPersona', 
                                                                                   'DepartamentoEmpresa']]))
data_afiliados.head()

In [ ]:
data_afiliados.columns[data_afiliados.isna().any()].tolist()

### Codificación de variables Categóricas

In [ ]:
categoricas

In [ ]:
data_afiliados=pd.get_dummies(data_afiliados, columns=categoricas, prefix=["cat_", "pir1_", "pir_2", "genero_","sec_", "civ_", "deptoP_", "mpioP_",
                                                              "grfm_", "DeptoEmp_", "CIIU_"])
data_afiliados.head(3)

## Modelo de regresion

In [ ]:
data_reg = data_afiliados[data_afiliados['Venta_Neta']>0]
data_reg.shape

In [ ]:
data_reg.columns[data_reg.isna().any()].tolist()

### Construccion de conjuntos Train y Test

In [ ]:
X = data_reg.drop(['Venta_Neta', 'IdPersona', 'IdEmpresa', 'Consume'], axis=1)
# y = np.log(data_reg['Venta_Neta'])
y = data_reg['Venta_Neta']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=31415)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LarsCV
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

models = {
    'rl': LinearRegression(normalize=True, n_jobs=-1)
    ,'ridge': RidgeCV(cv=10,alphas=(0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1))
    ,'lasso': LassoCV(cv=10,alphas=(0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1))
    ,'SGD': SGDRegressor()
    ,'LARS': LarsCV(n_jobs=-1)
    ,'Bay_Ridge': BayesianRidge()
    ,'AdaBoost': AdaBoostRegressor()
    ,'Bagging': BaggingRegressor(random_state=31415)
    ,'Trees': ExtraTreesRegressor(random_state=31415)
    ,'GradientBoosting': GradientBoostingRegressor(random_state=31415, n_estimators=500)
    ,'RandomForest': RandomForestRegressor(random_state=31415)
    ,'ExtraTrees': ExtraTreesRegressor(random_state=31415)
}

In [ ]:
for model in models.keys():
    models[model].fit(X_train, y_train)

In [ ]:
y_pred = pd.DataFrame(index=X_test.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

for model in models.keys():
    print(model, ": Error Cuadrático Medio: %.4f"
      % mean_squared_error(y_test,y_pred[model]))

#### CV sobre el Modelo final

La validación cruzada o cross-validation es una técnica utilizada para evaluar los resultados de un análisis estadístico y garantizar que son independientes de la partición entre datos de entrenamiento y prueba. Consiste en repetir y calcular la media aritmética obtenida de las medidas de evaluación sobre diferentes particiones. Se utiliza en entornos donde el objetivo principal es la predicción y se quiere estimar la precisión de un modelo que se llevará a cabo a la práctica.1​ Es una técnica muy utilizada en proyectos de inteligencia artificial para validar modelos generados.


<img align="center" width="500" height="300" src="https://upload.wikimedia.org/wikipedia/commons/1/18/Esquema_castell%C3%A0.jpg">



In [ ]:
from sklearn.model_selection import cross_val_score
FinalMod=models['Bay_Ridge']

n_cv=10
score=-(cross_val_score(FinalMod, X_train, y_train, cv=n_cv, scoring='neg_mean_squared_error'))
print("train MSE: \n", pd.Series(score).describe(),"\n")

score=-(cross_val_score(FinalMod, X_test, y_test, cv=n_cv, scoring='neg_mean_squared_error'))
print("test MSE: \n", pd.Series(score).describe())

## Modelo de Clasificacion

### Construccion de conjuntos Train y Test

In [ ]:
X = data_afiliados.drop(['Venta_Neta', 'IdPersona', 'IdEmpresa', 'Consume'], axis=1)
# y = np.log(data_reg['Venta_Neta'])
y = data_afiliados['Consume']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=101)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
sns.countplot(x="Consume", data=data_afiliados)

In [ ]:
y_train.mean()

## Balanceo de Muestra

Se procede a hacer un [SMOTE](https://arxiv.org/pdf/1106.1813.pdf) Synthetic Minority Over-sampling Technique

<img align="center" width="500" height="300" src="https://www.researchgate.net/publication/287601878/figure/fig1/AS:316826589384744@1452548753581/The-schematic-of-NRSBoundary-SMOTE-algorithm.png">

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
sm = SMOTE(random_state=101)
X_res, y_res = sm.fit_sample(X_train, y_train)

In [ ]:
print(X_res.shape, y_res.shape)

In [ ]:
sns.countplot(y_res)

In [ ]:
y_res.mean()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


models = {'lr': LogisticRegression()
          ,'dt': DecisionTreeClassifier()
          ,'rf': RandomForestClassifier(random_state=31415, n_jobs=-1, n_estimators=1000)
          ,'gb': GradientBoostingClassifier(random_state=31415)
          ,'ab': AdaBoostClassifier(random_state=31415)
          ,'nn': MLPClassifier(alpha=1, max_iter=5000)
          ,'kn': KNeighborsClassifier(5)
          ,'qd': QuadraticDiscriminantAnalysis()
         }

In [ ]:
for model in models.keys():
    models[model].fit(X_res, y_res)

In [ ]:
y_pred = pd.DataFrame(index=X_test.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score, roc_auc_score

for model in models.keys():
    print(model,'accuracy',accuracy_score(y_test,y_pred[model]))
    print(model,'ROC',roc_auc_score(y_test,y_pred[model]))    
    print(model,'precision',precision_score(y_test,y_pred[model]))
    print(model,'recall',recall_score(y_test,y_pred[model]))
    print(model,'f1 score', f1_score(y_test,y_pred[model]))